In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from Evaluation import get_metrics
%load_ext autoreload
%autoreload 2

---
## Read Data

In [2]:
from Data_prep import leave_last_out

In [3]:
path = 'C:/Users/robin.opdam/Google Drive/Thesis (Msc)/Thesis_shared_files/'
# path = '/Users/Robin/Google Drive/Thesis (Msc)/Thesis_shared_files/'
# path = '../'

In [4]:
# data_path = 'Data/Amazon/'
# file_name = 'am_20k_users'
# file_name = 'am_like_ml'

data_path = 'Data/ML/'
file_name = 'ml_1m'

In [5]:
df = pd.read_pickle(path + data_path + file_name)
total_items = df.item_id.nunique()
total_users = df.user_id.nunique()

In [6]:
# MF based Data
train_set_mf = pd.read_pickle(path + data_path + file_name + '_train_mf')
val_set_mf = pd.read_pickle(path + data_path + file_name + '_val_mf')
test_set_mf = pd.read_pickle(path + data_path + file_name + '_test_mf')

In [7]:
# Eval
steps = 5
rank_at = 20
n_runs = 30

## Top 5 GS results

In [8]:
pd.read_pickle(path + 'Logs/GS/' + file_name + '_ncf_gs_log').sort_values('test_rec@10', ascending=False).iloc[:5]

,val_rec@10,learning_rate,train_loss,val_auc,p,q,train_time,total_test_rec,test_rec@10,nolf,n_iterations,sample_size,seed,alpha,rho,sigma,reg_user,reg_item,file
5,"[0.014, 0.04, 0.04, 0.05, 0.048, 0.048, 0.042,...","[0.03, 0.033, 0.036300000000000006, 0.03993000...","[0.6928840096669814, 0.6885358726376268, 0.611...",[],0,0,305.445064,0.328,0.078,8,25,299612.7,1234,0.03,1.1,0.5,0,0,ml_1m
41,"[0.046, 0.046, 0.054, 0.048, 0.048, 0.058, 0.0...","[0.08, 0.08800000000000001, 0.0968000000000000...","[0.6681123964756175, 0.3958474435381411, 0.310...",[],0,0,297.027570,0.352,0.077,8,25,299612.7,1234,0.08,1.1,0.5,0.001,0.001,ml_1m
45,"[0.046, 0.046, 0.06, 0.056, 0.052, 0.054, 0.06...","[0.1, 0.11000000000000001, 0.12100000000000002...","[0.6242018303224871, 0.3482956618510598, 0.293...",[],0,0,271.964864,0.345,0.076,8,25,299612.7,1234,0.10,1.1,0.5,0.001,0.001,ml_1m
37,"[0.028, 0.046, 0.052, 0.046, 0.048, 0.046, 0.0...","[0.05, 0.05500000000000001, 0.0605000000000000...","[0.6919734206500515, 0.5955196889772099, 0.377...",[],0,0,298.123402,0.342,0.075,8,25,299612.7,1234,0.05,1.1,0.5,0.001,0.001,ml_1m
1,"[0.01, 0.01, 0.012, 0.016, 0.03, 0.036, 0.046,...","[0.01, 0.011000000000000001, 0.012100000000000...","[0.6932482913189884, 0.6927994985776338, 0.692...",[],0,0,312.357428,0.325,0.073,8,25,299612.7,1234,0.01,1.1,0.5,0,0,ml_1m


---
# NCF
- ML: specify which GS result to use

In [23]:
GMF_params = {
    'learning_rate': 0.0005,
    'batch_size': 256,
    'nolf': 8,
    'regs': [0.000001,0.000001],
    'epochs': 10,
    'sample_size': len(train_set_mf),#int(0.5*len(train_set.user_id.unique())),
    'num_neg': 4,
    'weights_dir': f'../Logs/weights/GMF_weights_{file_name}/GMF_weights',
    'optimizer':'Adam'
}

MLP_params = {
    'learning_rate': 0.0005,
    'batch_size': 256,
    'layers': [16,32,16,8],
    'reg_layers': [0.00001,0.00001,0.00001,0.00001],   #ML: [+-0.0005,0,0,0],
    'epochs': 10,
    'sample_size': len(train_set_mf),#int(0.5*len(train_set.user_id.unique())),
    'num_neg': 4,
    'weights_dir': f'../Logs/weights/MLP_weights_{file_name}/MLP_weights',
    'optimizer':'Adam'
}

NeuMF_params = {
    'learning_rate': 0.0005,
    'batch_size': 256,
    'layers': [16,32,16,8],
    'reg_layers': [0.00001,0.00001,0.00001,0.00001],#[0.0001, 0.0001, 0.0001, 0.0001],
    'reg_mf': [0.000001,0.000001],
    'nolf': 8,
    'epochs': 25,
    'sample_size': len(train_set_mf),#int(0.5*len(train_set.user_id.unique())),
    'num_neg': 4,
    'weights_dir': f'../Logs/weights/NeuMF_weights_{file_name}/NeuMF_weights',
    'optimizer':'Adam'
}

In [24]:
from NCF import NCF
ncf = NCF(total_users, total_items, GMF_params, MLP_params, NeuMF_params)
ncf.build_GMF_model()
ncf.build_MLP_model()
ncf.build_NeuMF_model()

In [21]:
sample_path = path + data_path + 'Samples/' + file_name + '_samples_' + NeuMF_params['num_neg'] + '_neg/'
samples = ncf.load_samples(sample_path, file_name + '_sample', NeuMF_params['epochs'])

100% |########################################################################|


### Pre-Train GMF and MLP to conversion

In [ ]:
GMF_val_metrics = ncf.train_model('GMF', samples, train_set=train_set, val_set=val_set, verbose=1)

In [ ]:
MLP_val_metrics = ncf.train_model('MLP', samples, train_set=train_set, val_set=val_set, verbose=1)

### Run NeuMF with GMF and MLP init

In [26]:
NCF.use_pretrain_model()
NCF.NeuMF_params['optimizer'] = 'SGD'

In [ ]:
for i in range(num_runs):
    NeuMF_val_metrics = NCF.train_model('NeuMF', samples, train_set=train_set_mf, val_set=val_set_mf, verbose=1)
    ranked_df_neumf = NCF.get_predictions('NeuMF', train_set=train_set_mf, test_set=test_set_mf)
    neumf_metrics = get_metrics(ranked_df_neumf, stats=False)

In [28]:
ranked_df_neumf = NCF.get_predictions('NeuMF', train_set=train_set_mf, test_set=test_set_mf)
neumf_metrics = get_metrics(ranked_df_neumf, stats=False)
neumf_metrics

100% |########################################################################|


,rank_at,hitcounts,recall,precision,ndcg
0,1,9,0.009,0.009000,0.009000
1,5,19,0.019,0.003800,0.013959
2,10,24,0.024,0.002400,0.015539
3,15,32,0.032,0.002133,0.017668
4,20,38,0.038,0.001900,0.019069
